In [1]:
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader



In [2]:
class ECGDataset(Dataset):
    def __init__(self, file_paths, labels, transform = None, mode = 'random'):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform
        self.mode = mode
    def __len__(self):
        return len(self.file_paths)
    def apply_thresholds(self, img):
        _, th_binary = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)
        _, th_otsu = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        _, th_adaptive = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_, cv2.THRESH_BINARY, 11, 2)
        return [th_binary, th_otsu, th_adaptive]
    def __getitem__(self, idx):
        img = cv2.imread(self.file_paths[idx], cv2.IMREAD_GRAYSCALE)
        label = self.labels[idx]
        imgs = [img, cv2.medianBlur(img, 3)]
        variants = []
        for base_img in imgs:
            variants.append(base_img)
            variants.append(self.apply_thresholds(base_img))
        if self.mode == 'random':
            chosen_img = variants[np.random.randint(len(variants))]
            final_img = chosen_img
        else:
            final_img = np.stack(variants, axis = 0)
        if self.transform:
            final_img = self.transform(final_img)
        else:
            if final_img.ndim == 2:
                final_img = torch.from_numpy(final_img).float().unsqueeze(0) / 255.0
            elif final_img.ndim == 3:
                final_img = torch.from_numpy(final_img).float().unsqueeze(1) / 255.0
        return final_img, label

In [3]:
import os

def get_image_paths_and_labels(data_root):
    file_paths = []
    labels = []
    label_names = sorted(os.listdir(data_root))
    label_to_idx = {label: idx for idx, label in enumerate(label_names)}

    for label in label_names:
        label_folder = os.path.join(data_root, label)
        if not os.path.isdir(label_folder):
            continue
        for fname in os.listdir(label_folder):
            if fname.lower().endswith(('png', 'jpg', 'jpeg')):
                file_paths.append(os.path.join(label_folder, fname))
                labels.append(label_to_idx[label])
    return file_paths, labels, label_to_idx

data_root = 'data/sorted_images'
file_paths, labels, label_to_idx = get_image_paths_and_labels(data_root)

dataset = ECGDataset(file_paths, labels)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleECGCNN(nn.Module):
    def __init__(self, num_classes, resize_shape = (256, 256)):
        super(SimpleECGCNN, self).__init__()
        self.resize_shape = resize_shape
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        out_h, out_w = self.resize_shape[0] // 8, self.resize_shape[1] // 8
        self.fc1 = nn.Linear(128 * out_h * out_w, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = F.interpolate(x, size = self.resize_shape, mode = 'bilinear', align_corners = False)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

num_classes = 7
resize_shape = (256, 256)
model = SimpleECGCNN(num_classes, resize_shape)

In [5]:
import torch.optim as optim
from torch.utils.data import random_split

In [6]:
from sklearn.model_selection import train_test_split
train_paths, val_paths, train_labels, val_labels = train_test_split(
    file_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

In [7]:
resize_shape = (256, 256)
batch_size = 16

train_dataset = ECGDataset(train_paths, train_labels)
val_dataset = ECGDataset(val_paths, val_labels)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


In [8]:
num_classes = len(label_to_idx)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleECGCNN(num_classes, resize_shape).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-4)

In [1]:
import torch
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cuda')

In [2]:
a=torch.cuda.FloatTensor()

C:\Users\ferin\AppData\Local\Temp\ipykernel_18980\3024658294.py:1: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\tensor\python_tensor.cpp:80.)
  a=torch.cuda.FloatTensor()


In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs} | ")
    model.train()
    running_loss = 0.0
    correct = total = 0
    for imgs, lbls in train_loader:
        imgs, lbls = imgs.to(device), lbls.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, lbls)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
        _, predicted = outputs.max(1)
        total += lbls.size(0)
        correct += predicted.eq(lbls).sum().item()
    train_loss = running_loss / total
    train_acc = correct / total

    model.eval()
    val_running_loss = 0.0
    val_correct = val_total = 0
    with torch.no_grad():
        for imgs, lbls in val_loader:
            imgs, lbls = imgs.to(device), lbls.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, lbls)
            val_running_loss += loss.item() * imgs.size(0)
            _, predicted = outputs.max(1)
            val_total += lbls.size(0)
            val_correct += predicted.eq(lbls).sum().item()
    val_loss = val_running_loss / val_total
    val_acc = val_correct / val_total

    print(
        f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
        f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}"
    )

Epoch 1/10 | 
